<a href="https://colab.research.google.com/github/mekhi-woods/HiloCATsSN1991bg/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# START-UP
import os
import shutil
if os.path.exists('/content/HiloCATsSN1991bg') == True:
    shutil.rmtree('/content/HiloCATsSN1991bg')
    !git clone https://github.com/mekhi-woods/HiloCATsSN1991bg.git
else:
    !git clone https://github.com/mekhi-woods/HiloCATsSN1991bg.git

!pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple snpy
!pip install requests


In [ ]:
"""
Plotting 91bg like 1a to get an idea of what they look like
"""
import matplotlib.pyplot as plt
import numpy as np

FILTER_WHEEL = ['u', 'g', 'r', 'i', 'B', 'V0']

if __name__ == '__main__':
    KrisciunasPath = "/content/HiloCATsSN1991bg/targetLists/91bglike_justnames.txt"
    KrisciunasNames = np.genfromtxt(KrisciunasPath, dtype=str, delimiter=', ')

    allCPSPhot = "/content/HiloCATsSN1991bg/data/CSPdata/SN_photo.dat"
    allCPSPhotData = np.genfromtxt(allCPSPhot, dtype='str')

    names = allCPSPhotData[:,0]
    filters = allCPSPhotData[:,1]
    time = allCPSPhotData[:,2]
    light = allCPSPhotData[:,3]
    err = allCPSPhotData[:,4]

    plt.figure(figsize=(10,6))
    sigma = 1
    for tar in KrisciunasNames:
        for n in range(len(FILTER_WHEEL)):
            # output_names = names[(names == tar) & (filters == FILTER_WHEEL[n])]
            output_light = light[(names == tar) & (filters == FILTER_WHEEL[n])].astype('float64')
            output_time = time[(names == tar) & (filters == FILTER_WHEEL[n])].astype('float64') + 53000
            output_err = err[(names == tar) & (filters == FILTER_WHEEL[n])].astype('float64')
            plt.errorbar(output_time, output_light, yerr=output_err*sigma, fmt='o', label=FILTER_WHEEL[n])

        plt.title(tar); plt.xlabel('Time [MJD]'); plt.ylabel('Intensity [mag]')
        plt.gca().invert_yaxis()
        plt.legend()
        # plt.savefig('save\\'+str(tar)+'.png')
        plt.show()
        break


In [ ]:
"""
SNooPy fitting on CSP data
"""
import matplotlib.pyplot as plt
import numpy as np
import snpy
import os

def recover_dir():
    if os.path.exists('/content/saved_models') == False:
        os.mkdir('/content/saved_models')
    return

def snpy_fit(filePath, model='max_model', shapeParam='dm15', BandsToFit = ['B','g','r','i'], summarize=True):
    s = snpy.get_sn(filePath)

    # Set model parameters
    s.choose_model(model, stype=shapeParam)
    s.set_restbands() # Auto pick appropriate rest-bands

    # Fit data -- using David configurations
    fitargs = {'mangle':1,'calibration':0} # I don't remember what calibration is
    s.fit(BandsToFit,
          dokcorr=True,
          k_stretch=False,
          reset_kcorrs=True,
          **fitargs)

    # Show results
    if summarize:
        s.summary()
        # for param in s.parameters:
        #     print("{} = {} +/- {}".format(param, s.parameters[param], s.errors[param]))
    return s

def manual_muist_calc(s, M_B = -16.38):
    peakmB = s.get_max(['B'], use_model=True)[1][0]
    return peakmB - M_B

def KrisciunasFitting(paths, model='EBV_model2', shape='st', bands=['B','g','r','i'], output=False):
    n_old = [0,1,2,3,4,5,6,7,8,9,10,11,12]
    valid_n = [0,1,3,4,5,6,8,10,11,12]
    problem_children = [2,7,9]

    SNeDistances = {}

    for n in valid_n:
        tarName = paths[n][39:47].replace('_', '')
        tarSave = '/content/saved_models/'+tarName+'_'+model+'.snpy'
        print('[ '+str(n+1)+' / '+str(len(n_old))+'] Fiting data for '+tarName+'...')

        # Create/Retrieve Fit
        if os.path.exists(tarSave):
            s_n = snpy.get_sn(tarSave)
        else:
            s_n = snpy_fit(paths[n], model=model, shapeParam=shape, BandsToFit=bands, summarize=output) # Enter snpy fit function
            s_n.save(tarSave)

        # Pull SNooPY distance
        snpy_mu = s_n.get_distmod() # Nab paramaters from SNe objects)

        # Update dictionary
        SNeDistances.update({tarName: snpy_mu})

        # Print info
        print('Redshift:\t z = '+str(s_n.z))
        print('Distance: \t mu = '+str(round(snpy_mu, 4)))
        print('\n')
        plt.clf()
    return SNeDistances

if __name__ == '__main__':
    recover_dir()

    # Initalize File -- SN2005ke
    KrisciunasNames = np.genfromtxt("/content/HiloCATsSN1991bg/targetLists/91bglike_justnames.txt", dtype=str, delimiter=', ')
    filePaths = []
    for tar in KrisciunasNames:
        filePaths.append('/content/HiloCATsSN1991bg/data/CSPdata/'+str(tar)+'_snpy.txt')

    snpyDistances = KrisciunasFitting(paths=filePaths,
                                      model='EBV_model2',
                                      shape='st',
                                      bands=['B','g','r','i'],
                                      output=False)

    burnsData = np.genfromtxt('/content/HiloCATsSN1991bg/targetLists/burns+25table2ext.txt', skip_header=0, delimiter=' ', dtype=str)
    burnsDistances = {}
    for tar in np.stack((burnsData[:, 0], burnsData[:, 14]), axis=1):
        burnsDistances.update({tar[0]: float(tar[1])})

    # Plot Distance v. Distance
    print("Ploting differences in distance calculations...")
    plt.figure(figsize=(8,5))
    for x in snpyDistances:
        try:
            plt.scatter(burnsDistances[x], snpyDistances[x])
            plt.text(burnsDistances[x]+0.05, snpyDistances[x]-0.05, x, fontsize='xx-small')
            print(burnsDistances[x], snpyDistances[x])
        except KeyError:
            print(x, 'not found in Burns+25 Table 2.')
            pass
    plt.title("Burns v. SNooPy Distance"); plt.xlabel('Burns Distance'); plt.ylabel('SNooPy Distance')
    plt.savefig('burns_v_snpy_dist.png')
    plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import urllib.request
import requests
import os
from requests.auth import HTTPBasicAuth

API_TOKEN = "7f4e1dee8f52cf0c8cdaf55bf29d04bef4810fb4"

ATLAS_PATH = '/content/ATLAS'

def recover_dir():
    if os.path.exists(ATLAS_PATH) == False:
        os.mkdir(ATLAS_PATH)
    return

def main():

        pickle = np.load('tmp.npz', allow_pickle=True)
        data = pickle['data']

        data = requests.post(
            'https://star.pst.qub.ac.uk/sne/atlas4/api/objectlist/',
            headers={'Authorization': f'Token {API_TOKEN}'},
            data={'objectlistid':2}
        ).json()

        np.savez('tmp.npz', data=data)

        count = 0
        for d in data:
            if d['observation_status'] is not None and d['observation_status'].startswith('SN Ia') and '91bg' in d['observation_status']:
                print(d['atlas_designation'],d['observation_status'].replace(' ',''),d['ra'],d['dec'])
                count += 1

                ids = d['id']
                base_url = 'https://star.pst.qub.ac.uk/sne/atlas4/lightcurveforced/1161048951013729300/'
                new_url = base_url.replace('1161048951013729300/',str(ids))
                print(new_url)

                idfile = ATLAS_PATH+'/' + str(ids)+'.txt'
                if os.path.exists(idfile):
                    continue
                urllib.request.urlretrieve(str(new_url), str(idfile))
                print(idfile)

            if count > 300:
                break

def plotting():
    testPath = '/content/ATLAS/1000438990080559900.txt'
    data = np.genfromtxt(testPath, dtype=float, delimiter=',')
    t = data[:, 8]
    uJy = data[:, 24]
    plt.scatter(t, uJy, linewidth=0.1)
    plt.ylim(-250, 230)
    plt.show()



    return

if __name__ == '__main__':
    # recover_dir()
    # print (main())
    plotting()